<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/RandomForest/VariableSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

# Chargement des données

In [ ]:
!wget --no-check-certificate --content-disposition "https://raw.githubusercontent.com/AlexandreBourrieau/ML/main/Carnets%20Jupyter/RandomForest/prostate.csv"


Les données contiennent 102 observations de 6033 gènes. Dans cet ensemble, il y a 52 cancers sur les 102 patients :
 - 2 variables sont très importantes
 - Environ 20 variables sont de moyenne importance
 - Le reste des variables est de faible importance 

In [ ]:
df_data=pd.read_csv('prostate.csv')
df_data = df_data.set_index(df_data['Unnamed: 0'])
df_data

# Random Forest - Choix des paramètres

La caractérisation de l'importance des variables est cruciale. Cette caractéristique va dépendre :
 - Du ratio entre le nombre de variables p et le nombre d'observations n
 - Des paramètres définissant le modèle :
    - Nombre d'arbres construits par l'algorithme (ntree)
    - Nombre de variables testées à chaque division (mtry) 
 - De la présence de variables fortements corrélées

**1. Choix du nombre d'arbres**

On choisit le nombre d'arbres en prenant comme valeur de mtry la racine carrée du nombre de variables, puis on teste plusieurs valeurs en évaluant comment réduit l'OOB en fonction du nombre d'arbres générés.  
On choisit la valeur qui stabilise le minimum de l'erreur.

In [ ]:
X = df_data.drop(columns='y')
X = X.drop(columns='Unnamed: 0')
X

In [ ]:
Y = df_data['y'].apply(lambda x: 1 if (x=='cancer') else 0)
Y

RandomForestClassifier :
  - n_estimators : Nombre d'arbre dans la forêt (ntree)
  - oob_score : Sauvegarde du score OOB
  - max_samples : Nombre maximal d'observations à utiliser
  - max_features : Nombre maximum de divisions (mtry)

In [ ]:
# Informations sur les données
n = 102               # Nombre d'observations
p = 6033              # Nombre de variables

n_arbres_max = 5000

n_arbres = np.linspace(1,n_arbres_max,10).astype(np.int32)
mtry = np.sqrt(p).astype(np.int32)
OOB_err = []

for i in n_arbres:
  print("#Arbres : %d" %i)
  clf = RandomForestClassifier(n_estimators=i, bootstrap=True, oob_score=True, max_samples = n, max_features = mtry, n_jobs=-1)
  clf.fit(X,np.asarray(Y))
  OOB_err.append(1 - clf.oob_score_)

In [ ]:
plt.plot(n_arbres,OOB_err)

**2. Choix de mtry**

Pour choisir la valeur de mtry, on fait tourner Random Forest 10 fois avec dix valeurs de mtry différentes, puis on choisit celle pour laquelle le mtry est minimal et se stabilise.

In [ ]:
# Informations sur les données
n = 102               # Nombre d'observations
p = 6033              # Nombre de variables

n_arbres = 2000
mtry_0 = (np.sqrt(p)/2).astype(np.int32)

m_try = np.linspace(mtry_0,2000,10).astype(np.int32)

OOB_err = []

for i in m_try:
   print("mtry = %s" %i)
   clf = RandomForestClassifier(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features=i, n_jobs=-1)
   clf.fit(X,np.asarray(Y))
   OOB_err.append(1 - clf.oob_score_)

In [ ]:
plt.plot(m_try,OOB_err)

On choisit m_try = 1000

# Random Forest - Importance des variables

Permutation importance is a technique where we shuffle the values of a single column and run the model to see how the scores get affected. If the scores are affected greatly, then the feature is highly important to the model and if not, it does not add significant value to the model.

**1. Calcul de l'importance des variables avec la méthode Gini**

In [ ]:
from sklearn.inspection import permutation_importance

# Informations sur les données
n = 102               # Nombre d'observations
p = 6033              # Nombre de variables
n_arbres = 2000
m_try = 2011

clf = RandomForestClassifier(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features=m_try, n_jobs=-1)
clf.fit(X,np.asarray(Y))

In [ ]:
col_sorted_by_importance=clf.feature_importances_.argsort()
feat_imp = pd.DataFrame({'cols':X.columns[col_sorted_by_importance],'imps':clf.feature_importances_[col_sorted_by_importance]})
feat_imp

In [ ]:
!pip install plotly_express --upgrade -q

In [ ]:
import plotly_express as px
import plotly.offline as po

px.bar(feat_imp.sort_values(['imps'], ascending=False)[:25], x='cols', y='imps', labels={'cols':'column', 'imps':'feature importance'})

**2. Calcul de l'importance des variables avec la méthode des permutations**

In [ ]:
import random

def PermImportance(X, y, clf, metric, num_iterations=100):
    '''
    Calculates the permutation importance of features in a dataset.
    Inputs:
    X: dataframe with all the features
    y: array-like sequence of labels
    clf: sklearn classifier, already trained on training data
    metric: sklearn metric, such as accuracy_score, precision_score or recall_score
    num_iterations: no. of repetitive runs of the permutation
    Outputs:
    baseline: the baseline metric without any of the columns permutated
    scores: differences in baseline metric caused by permutation of each feature, dict in the format {feature:[diffs]}
    '''
    bar=progressbar.ProgressBar(max_value=len(X.columns))
    baseline_metric=metric(y, clf.predict(X))
    scores={c:[] for c in X.columns}
    for c in X.columns:
        X1=X.copy(deep=True)
        for _ in range(num_iterations):
            temp=X1[c].tolist()
            random.shuffle(temp)
            X1[c]=temp
            score=metric(y, clf.predict(X1))
            scores[c].append(baseline_metric-score)
        bar.update(X.columns.tolist().index(c))
    return baseline_metric, scores

In [ ]:
from sklearn.inspection import permutation_importance

# Informations sur les données
n = 102               # Nombre d'observations
p = 6033              # Nombre de variables
n_arbres = 2000
m_try = 2011

clf = RandomForestClassifier(n_estimators=n_arbres, bootstrap=True, oob_score=True, max_features=m_try, n_jobs=-1)
clf.fit(X,np.asarray(Y))


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import progressbar

baseline, scores=PermImportance(X, np.asarray(Y), clf, recall_score, num_iterations=10)

In [ ]:
X.transpose().index

In [ ]:
fig, ax = plt.subplots()
foret_importances[3:20].plot.bar(ax=ax)
ax.set_title("Importances")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()